In [10]:
from minio.error import S3Error
import fitz  # PyMuPDF
from datetime import datetime
from io import BytesIO
import pandas as pd
import re
import psycopg2
import import_ipynb
from Utils import conectar_minio,transformar_df_lab_Jimenez, generar_ruta_fecha,extraer_archivo_minio, insert_into_info_lab, cargar_dfpaciente

In [11]:
minio_client = conectar_minio(
    endpoint='localhost:9000',
    access_key='crosero',
    secret_key='12345678',
    secure=False  # Usar HTTPS
)

Conexión exitosa a MinIO en localhost:9000
Buckets disponibles: ['yachay', 'yachay-bronze', 'yachay-landing']


In [12]:
ruta_fecha = generar_ruta_fecha()  

In [13]:
import pandas as pd

def procesar_laboratorios_jimenes(minio_client, ruta_fecha, connection_params):
    """
    Procesa y carga datos de los laboratorios Jimenes Ipiales y Pasto desde MinIO a PostgreSQL
    y devuelve un único DataFrame unificado.
    """
    
    # 1. Configuración de rutas para ambos laboratorios
    config_laboratorios = {
        "Ipiales": {
            "bucket": "yachay-bronze",
            "ruta": f"laboratorios/{ruta_fecha}/laboratorio Jimenes Ipiales/laboratorio_jimenes_ipiales.csv",
            "tipo_archivo": "csv"
        },
        "Pasto": {
            "bucket": "yachay-bronze",
            "ruta": f"laboratorios/{ruta_fecha}/laboratorio Jimenez Pasto/laboratorio_jimenes_pasto.csv",
            "tipo_archivo": "csv"
        }
    }
    
    # 2. Lista para guardar todos los DataFrames transformados
    lista_df = []
    
    # 3. Procesamiento para cada laboratorio
    for lab_name, config in config_laboratorios.items():
        try:
            print(f"\nProcesando Laboratorio Jimenes {lab_name}...")
            
            # Extraer archivo
            print(f"Extrayendo archivo: {config['ruta']}")
            df_datos = extraer_archivo_minio(
                minio_client=minio_client,
                bucket_name=config["bucket"],
                ruta_archivo=config["ruta"],
                tipo_archivo=config["tipo_archivo"]
            )
            
            # Transformar datos
            print("Transformando datos...")
            df_transformado = transformar_df_lab_Jimenez(df_datos)
            #print(df_transformado.head())  # Mostrar las primeras filas
            
            # Insertar en PostgreSQL
            print("Insertando en base de datos...")
            insert_into_info_lab(df_transformado, connection_params)
            
            # Guardar DataFrame en lista
            lista_df.append(df_transformado)
            
            print(f"Laboratorio Jimenes {lab_name} procesado con éxito!")
            
        except Exception as e:
            print(f"\nError procesando Laboratorio Jimenes {lab_name}:")
            print(str(e))
            continue

    # 4. Unificar todos los DataFrames en uno solo
    if lista_df:
        df_unificado = pd.concat(lista_df, ignore_index=True)
        print("\nTodos los laboratorios procesados y unificados!")
        return df_unificado
    else:
        print("\nNo se pudo procesar ningún laboratorio.")
        return pd.DataFrame()  # Retorna DataFrame vacío si fallaron todos


In [14]:
connection_params = {
    'dbname': 'udenaryachay',
    'user': 'postgres',
    'password': '12345678',
    'host': 'localhost',
    'port': 5432
}

In [15]:
df_unificado = procesar_laboratorios_jimenes(minio_client, ruta_fecha, connection_params)



Procesando Laboratorio Jimenes Ipiales...
Extrayendo archivo: laboratorios/2025/05/06/laboratorio Jimenes Ipiales/laboratorio_jimenes_ipiales.csv


Transformando datos...
Insertando en base de datos...
Se insertaron 5 filas en la tabla 'infolab'.
Laboratorio Jimenes Ipiales procesado con éxito!

Procesando Laboratorio Jimenes Pasto...
Extrayendo archivo: laboratorios/2025/05/06/laboratorio Jimenez Pasto/laboratorio_jimenes_pasto.csv
Transformando datos...
Insertando en base de datos...
Se insertaron 5 filas en la tabla 'infolab'.
Laboratorio Jimenes Pasto procesado con éxito!

Todos los laboratorios procesados y unificados!


In [16]:
print(df_unificado.columns)

Index(['archivo', 'Ruta Completa', 'Numero de Informe', 'Nombre del Paciente',
       'id_paciente', 'Edad', 'Telefono', 'fecha_toma_muestra',
       'fecha_ingreso', 'fecha_informe', 'entidad', 'eps', 'servicio',
       'muestra_remitida', 'descripcion_macroscopica',
       'descripcion_microscopica', 'diagnostico', 'comentario',
       'TipoIdentificacion', 'NumeroIdentificacion', 'fuente'],
      dtype='object')


In [17]:
# Crear el DataFrame df_pacientes con las columnas renombradas
df_pacientes = df_unificado[['Nombre del Paciente', 'TipoIdentificacion', 'NumeroIdentificacion', 'Edad', 'id_paciente']].copy()

# Renombrar las columnas
df_pacientes = df_pacientes.rename(columns={
    'Nombre del Paciente': 'nombre1',
    'TipoIdentificacion': 'tipo_de_identificacion',
    'NumeroIdentificacion': 'identificacion',
    'Edad': 'edad',
    'id_paciente': 'id_paciente'
})

df_pacientes = df_pacientes.drop_duplicates(subset=['id_paciente'], keep='first')

print(df_pacientes.head())

                           nombre1 tipo_de_identificacion identificacion  \
0     GRACIELA ESTHER LOZA ALVAREZ                     CC       27355204   
1    YULI VIVIANA CALPA ARCINIEGAS                     CC     1085915998   
2    DANY PAOLA CHACUA PUERCHAMBUD                     CC     1085945182   
3     STELLA ISAURA YANDUN PANTOJA                     CC       37120149   
4  SANDRA VIVIANA CHAMORRO ESCOBAR                     CC     1085918055   

   edad   id_paciente  
0  59.0    CC27355204  
1  33.0  CC1085915998  
2  25.0  CC1085945182  
3  44.0    CC37120149  
4  32.0  CC1085918055  


In [18]:
cargar_dfpaciente(df_pacientes,'paciente', connection_params)

Insertados 0 registros nuevos en paciente.


(True, 0)